### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpReadFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.exp.ListExp;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
Key key = new Key("test", "testset", "key1");
WritePolicy wPolicy = new WritePolicy();
Bin b0 = new Bin("b00", Value.get(28)); //8byte value
Bin b1 = new Bin("b01", Value.get("A quick fox jumps right over the lazy dog.")); 
List<Value> myList = new ArrayList<Value>();
Bin listData = new Bin("data", Value.get(myList));
//Note: Adding empty bin "data" so that list size expression can execute to start with.

client.put(wPolicy, key, b0, b1, listData);
//client.put(wPolicy, key, b0, b1, b2, b3, b4, b5, b6, b7,b8, b9, b10);
System.out.println("Record: "+ client.get(null, key)); //check record got inserted

//Using expressions to read record metadata in PI that has size of record on device
Expression recSizeExp = Exp.build(Exp.deviceSize());
Record record = client.operate( wPolicy, key,   
          ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT) 
         );
System.out.println("Record Size in bytes: " + record.getValue("size"));


//WritePolicy wPolicy = new WritePolicy();
// Filter expression is part of the policy object, supplied when calling 'operate'
boolean bOvr = false;
int iMaxListSize = 3000;

//For testing
wPolicy.filterExp = Exp.build(Exp.or(
    Exp.lt(ListExp.size(Exp.listBin("data")),Exp.val(iMaxListSize)), 
    Exp.val(bOvr))
    );

//Don't need override flag otherwise.
//wPolicy.filterExp = Exp.build(Exp.lt(ListExp.size(Exp.listBin("data")),Exp.val(iMaxListSize)));

// Want it to fail when filterExp = false
wPolicy.failOnFilteredOut = true;

try {
    while(true) {
        List<Value> values = new ArrayList<>();
        
        for (int i = 0; i < 500; i++) {
            values.add(Value.get("111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111"));
        }
        Expression recSizeExp = Exp.build(Exp.deviceSize());
        Record r1 = client.operate(wPolicy, key, 
                    ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), 
                    ListOperation.appendItems("data", values, null));
        System.out.println("Record Size in bytes: text " + r1.getValue("size"));      
        Record record = client.operate( null, key, 
                        ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), 
                        ListOperation.size("data", null)  //<-- gives number of elements in list
                        );       
        
        System.out.println("Record Size in bytes: text " + record.getValue("size"));        
        System.out.println("Bin Array (each element = #128 bytes) Size: " + record.getValue("data"));

        //Thread.sleep(5);  // Tried some sleep in the loop in my initial test with i<1 instead of i<500

    }
} catch (AerospikeException e){
  int rc = e.getResultCode();
  if(rc == ResultCode.FILTERED_OUT){
    System.out.println("Array Limit Exceeded. ==> Split the Record.");
  }
  else{
    System.out.println("Operate error. Resultcode: "+ ResultCode.getResultString(rc));
  }
    
}

System.out.println("DONE.");

//catch (Exception e) {
//    
//      e.printStackTrace();
//      
//}

Record: (gen:1),(exp:450847402),(bins:(b00:28),(b01:A quick fox jumps right over the lazy dog.),(data:[]))
Record Size in bytes: 128
Record Size in bytes: text 128
Record Size in bytes: text 67632
Bin Array (each element = #128 bytes) Size: 500
Record Size in bytes: text 67632
Record Size in bytes: text 135136
Bin Array (each element = #128 bytes) Size: 1000
Record Size in bytes: text 135136
Record Size in bytes: text 202624
Bin Array (each element = #128 bytes) Size: 1500
Record Size in bytes: text 202624
Record Size in bytes: text 270128
Bin Array (each element = #128 bytes) Size: 2000
Record Size in bytes: text 270128
Record Size in bytes: text 337632
Bin Array (each element = #128 bytes) Size: 2500
Record Size in bytes: text 337632
Record Size in bytes: text 405136
Bin Array (each element = #128 bytes) Size: 3000
Array Limit Exceeded. ==> Split the Record.
DONE.


# Cleanup

In [5]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"